In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ast
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data=pd.read_csv('/Users/youziya/Library/Group Containers/UBF8T346G9.OneDriveStandaloneSuite/OneDrive - York University.noindex/OneDrive - York University/MBAN 6090 Analytics Project/resume_data_240112_2.csv')

In [3]:
pip install transformers torch


You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


: 

In [ ]:
def preprocess_text(text):
    # Example: Replace new line characters and multiple spaces
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = ' '.join(text.split())
    return text

sample_text = preprocess_text(sample_resume_text)


In [ ]:
def extract_entities(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs).logits
    predictions = torch.argmax(outputs, dim=2)

    entities = []
    for token, prediction in zip(inputs.tokens(), predictions[0].numpy()):
        entity = model.config.id2label[prediction]
        if entity != "O":  # O means no entity
            entities.append((token, entity))
    return entities

entities = extract_entities(sample_text)
print(entities)


In [ ]:
import re

def extract_email_phone(text):
    email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    phone_regex = r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}'

    emails = re.findall(email_regex, text)
    phones = re.findall(phone_regex, text)

    return emails, phones

emails, phones = extract_email_phone(sample_text)
print("Emails:", emails)
print("Phones:", phones)


In [ ]:
# Assuming entities contains your NER extracted entities
# and you've extracted emails and phones as lists

resume_data = {
    "Email": emails[0] if emails else None,
    "Phone": phones[0] if phones else None,
    # Add more fields as necessary
}

print(resume_data)


In [4]:
data=pd.read_csv('/Users/youziya/Library/Group Containers/UBF8T346G9.OneDriveStandaloneSuite/OneDrive - York University.noindex/OneDrive - York University/MBAN 6090 Analytics Project/resume_data_240112_2.csv')

In [5]:
# Checking for duplicate rows
duplicate_rows = data.duplicated().sum()

# Text cleaning for 'Resume' and 'Resume_i' columns
# Removing special characters and extra spaces, converting to lowercase
data['Resume_cleaned'] = data['Resume'].str.replace('[^a-zA-Z0-9\s]', '', regex=True).str.lower().str.strip()
data['Resume_i_cleaned'] = data['Resume_i'].str.replace('[^a-zA-Z0-9\s]', '', regex=True).str.lower().str.strip()

# Checking for missing values
missing_values = data.isnull().sum()

duplicate_rows, missing_values


(0,
 Title               0
 Raw_html            2
 Resume              2
 Resume_i            2
 Resume_cleaned      2
 Resume_i_cleaned    2
 dtype: int64)

In [6]:
# Removing rows with missing values
data_cleaned = data.dropna()

# Checking the shape of the dataset after removing missing values
new_shape = data_cleaned.shape

new_shape



(6199, 6)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming we are using TF-IDF for text vectorization
vectorizer = TfidfVectorizer(max_features=1000)  # Limiting to 1000 features for simplicity

# Applying TF-IDF to the 'Resume_cleaned' column
tfidf_matrix = vectorizer.fit_transform(data['Resume_cleaned'])

# Creating a DataFrame for the TF-IDF features
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Displaying the first few rows of the TF-IDF DataFrame
tfidf_df.head()


ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

def extract_experience(resume_text):
    doc = nlp(resume_text)
    organizations = set()
    dates = set()

    for ent in doc.ents:
        if ent.label_ == "ORG":
            organizations.add(ent.text)
        elif ent.label_ in ["DATE", "TIME"]:
            dates.add(ent.text)
    
    return {"organizations": list(organizations), "dates": list(dates)}

# Apply to your resume data
extracted_info = your_dataframe['Resume_column'].apply(extract_experience)
